<h1>
Problema das Rainhas
</h1>

Em um tabuleiro 𝑛×𝑛, uma rainha é colocada em um quadrado, irá dominar todos os quadrados que estiverem na mesma linha, coluna e diagonais. A ideia por trás deste probelma é achar a quantidade mínima de rainhas necessárias para dominar o tabuleiro inteiro. Dominar, neste problema, significa cobrir todos os quadrados possíveis sendo atacados por rainhas incluindo aqueles onde as rainhas se encontram.

Primeiro, vamos começar definindo o tabuleiro como é feito no artigo. Vamos começar por um tabuleiro 4x4, igual começa no artigo:

In [182]:
import numpy as np
N = 8
chessBoard = np.array([x+1 for x in range(N*N)])

chessBoard = chessBoard.reshape(N,N)
print(chessBoard)

[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]


Agora, criar a função para verificar quadrados do tabuleiro que estão sendo dominados por todas as rainhas e retornar os números que estão sendo dominados:

In [183]:
def dominatedSet(queens, chessBoard, verbose=True):
    S = set()
    N = chessBoard.shape[0]
    for queen in queens:
        queenBox = queen
        queenPosition = np.where(chessBoard == queenBox)
        i = queenPosition[0][0]
        j = queenPosition[1][0]
        if verbose: 
            print("posição matricial: (", i,j, ")" , 'valor: ', queenBox)

        if(verbose):
            # quadrados dominados na horizontal:
            print(chessBoard[i,:])
            # quadrados dominados na vertical:
            print(chessBoard[:,j])
       
        S.update(chessBoard[i,:])
        S.update(chessBoard[:,j])
        # diagonal
        for d in range(-N+1,N):
            diag = chessBoard.diagonal(d)
            invertDiag = np.fliplr(chessBoard).diagonal(d)
            if queenBox in diag:
                if verbose:
                    print(diag)
                S.update(diag)
            if queenBox in invertDiag:
                if verbose:
                    print(invertDiag)
                S.update(invertDiag)
        # diagonal invertida
       # print(np.fliplr(chessBoard).diagonal(0))
    return S
        

O código abaixo vai então gerar a área de dominância para uma única rainha na posição (3,3) do tabuleiro, ou também podemos dizer, no valor 28 da matriz. 

<b>Para ficar mais claro, o número do quadrado em que a rainha se encontra nós vamos chamar de posição numérica. Então o quadrado 28, em que essa rainha se encontra, é sua posição numérica. A dupla (3,3) e todas as outras nós vamos chamar de posição matricial </b>

In [184]:
S = dominatedSet([28], chessBoard)
print(S)

posição matricial: ( 3 3 ) valor:  28
[25 26 27 28 29 30 31 32]
[ 4 12 20 28 36 44 52 60]
[ 1 10 19 28 37 46 55 64]
[ 7 14 21 28 35 42 49]
{1, 4, 7, 10, 12, 14, 19, 20, 21, 25, 26, 27, 28, 29, 30, 31, 32, 35, 36, 37, 42, 44, 46, 49, 52, 55, 60, 64}


A função que mede o quão próximo estamos de uma boa solução no artigo é dada por:

$ f(x) = {|S| \div |G|}  $

Nesta função, se o resultado for menor que 1, temos que existem alguns ou pelo menos 1 quadrado que não está numa área de dominância das rainhas. Se for 1, significa que as rainhas dominaram todos os quadrados do tabuleiro. 

No código, calcularemos essa função assim:

In [185]:
len(S)/ (N*N) 

0.4375

<h2>Codificação dos indivíduos </h2>

O conjunto de indivíduos será representado por uma matriz onde cada linha é um solução candidata e as colunas são posições das rainhas no tabuleiro para aquela solução. As posições das rainhas serão dadas pelos valores das posições numéricas definidas como um binário de 8 dígitos.
Seria Assim:


In [186]:
rainha1 = format(23,'08b')
rainha2 = format(43,'08b')
rainha3 = format(12,'08b')
rainha4 = format(10,'08b')

np.matrix([[rainha1, rainha2], [rainha3, rainha4]])

matrix([['00010111', '00101011'],
        ['00001100', '00001010']], dtype='<U8')

No caso, a primeira solução candidata tem as rainhas nas posições numéricas 23 e 43, e segunda solução candidata tem rainhas nas posições 12 e 10

<h2>Geração inicial de indivíduos </h2>

A primeira geração consiste de 100 indivíduos (soluções candidatas) geradas aleatoriamente, mantendo a certeza de que em cada possível solução as rainhas estão em posições diferentes. Vamos fazer uma função que faz isso. A função vai receber como parâmetro o número de rainhas que queremos para as soluções possíveis e a dimensão do tabuleiro.

In [187]:
from random import seed
from random import randint

seed(3)

value = randint(0, N*N)
value

def gen_individuals(N_queens, N, N_individuals): 
    A = np.empty((1,N_queens), dtype='str')
    for i in range (0,N_individuals):
        newRow = []
        for j in range (0,N_queens):
            randVal = randint(1,N*N)
            randInBin = format(randVal,'08b')
            while randInBin in newRow: 
                randVal = randint(1,N*N)
                randInBin = format(randVal,'08b')
            newRow.append(randInBin)
        A = np.vstack([A, newRow])
    # apaga a primeira linha que é gerada na inicialização da matriz
    A = np.delete(A,0, 0)
    return A



Vendo abaixo uma amostra de dois indivíduos. Cada linha é uma solução possível e essas soluções possuem duas rainhas. A função também permite que a gente decida quantos indivíduos queremos gerar para a matriz A (número de linhas) assim como quantas rainhas pode ter cada indivíduo (número de colunas)

In [188]:
N_queens = 2
A = gen_individuals(N_queens,N,100)
print(A[0])
print(A[99])

['00010001' '00110000']
['00011111' '00000011']


Já vou deixar pronta uma função que retorna o fitness do indivíduo. Sabemos que só os 50% melhores devem ser selecionados para a próxima geração.

In [189]:
def darwinize_individual(ind, chessBoard):
    indInInteger = []
    N = chessBoard.shape[0]
    for queen in ind:
        inIntQueen = int(queen,2)
        indInInteger.append(inIntQueen)
    S = dominatedSet(indInInteger, chessBoard, verbose=False)
    ## o jeito de calcular o fitness como foi dito anteriormente
    return len(S)/ (N*N) 
    

darwinize_individual(A[0], chessBoard)
fitness_values = np.empty((1), dtype='int64')
# uma coisa que dá pra ver é que os individuos passam todos da metade de dominância do tabuleiro
for indi in A:
    fit = darwinize_individual(indi, chessBoard)
    fitness_values = np.vstack([fitness_values, fit])
    if fit < 0.5:
        print(indi)
fitness_values = np.delete(fitness_values,0, 0)  




In [190]:
def maisAdaptados (pop, fitness, n_ind, n_queens):

    # Selecionará os 50% individuos com maior fitness

    ind = np.empty((1,n_queens), dtype='str')
    fit = np.empty((1), dtype='str')

    for i in range(n_ind):
        max_fitness_idx = np.argmax(fitness)
        fit = np.vstack([fit,fitness[max_fitness_idx]])
        ind = np.vstack([ind, pop[max_fitness_idx]])
        fitness[max_fitness_idx] = -99999999999
    ind = np.delete(ind,0, 0)
    fit = np.delete(fit,0,0)
    ind = ind[0:int(n_ind/2)]
    fit = fit[0:int(n_ind/2)]
    return ind,fit

   

In [191]:

B, fit_B = maisAdaptados(A, fitness_values, 100, 2 )



In [192]:
from random import uniform, random, choice, sample

def seleciona_roulette_wheel(pop, fitness):
  y = fitness.astype(np.float)
  Soma = np.sum(y)
  Prob_escolhido = np.empty((1), dtype='float')
  for i in y:   
    Prob_escolhido = np.vstack([Prob_escolhido, i/Soma])
    
  Prob_escolhido = np.delete(Prob_escolhido,0,0) 
  Prob_escolhido = np.transpose(Prob_escolhido) 
  indices = np.random.choice(len(pop),size= 2 ,p=Prob_escolhido.flatten())
  ind = pop[indices]
  return ind

  
ind = seleciona_roulette_wheel(B, fit_B)
print (ind)  



[['00101100' '00001011']
 ['00100010' '00011110']]
